All quotes below are excerpted from https://toadstyle.org/cryptopals/59.txt

---
```
59. Elliptic Curve Diffie-Hellman and Invalid-Curve Attacks

I'm not going to show you any graphs - if you want to see one, you can
find them in, like, every other elliptic curve tutorial on the
internet. Personally, I've never been able to gain much insight from
them.

They're also really hard to draw in ASCII.

The key thing to understand about elliptic curves is that they're a
setting analogous in many ways to one we're more familiar with, the
multiplicative integers mod p. So if we learn how certain primitive
operations are defined, we can reason about them using a lot of tools
we already have in our utility belts.

Let's dig in. An elliptic curve E is just an equation like this:

    y^2 = x^3 + a*x + b

The choice of the a and b coefficients defines the curve.

We'll use the notation GF(p) to talk about a finite field of size
p. (The "GF" is for "Galois field", another name for a finite field.)
When we take a curve E over field GF(p) (written E(GF(p))), what we're
saying is that only points with both x and y in GF(p) are valid.

For example, (3, 6) might be a valid point in E(GF(7)), but it
wouldn't be a valid point in E(GF(5)); 6 is not a member of GF(5).

...

Okay: if these points are going to form a group analogous to the
multiplicative integers mod p, we need to have an analogous set of
primitive functions to work with them.

1. In the multiplicative integers mod p, we combined two elements by
   multiplying them together and taking the remainder modulo p.

   We combine elliptic curve points by adding them. We'll talk about
   what that means in a hot second.

2. We used 1 as a multiplicative identity: y * 1 = y for all y.

   On an elliptic curve, we define the identity O as an abstract
   "point at infinity" that doesn't map to any actual (x, y)
   pair. This might feel like a bit of a hack, but it works.

   On the curve, we have the straightforward rule that P + O = P for
   all P.

   In your code, you can just write something like O := object(),
   since it only ever gets used in pointer comparisons. Or you can use
   some sentinel coordinate that doesn't satisfy the curve equation;
   (0, 1) is popular.

3. We had a modinv function to invert an integer mod p. This acted as
   a stand-in for division. Given y, it finds x such that y * x = 1.

   Inversion is way easier in elliptic curves. Just flip the sign on
   y, and remember that we're in GF(p):

       invert((x, y)) = (x, -y) = (x, p-y)

   Just like with multiplicative inverses, we have this rule on
   elliptic curves:

       P + (-P) = P + invert(P) = O

Incidentally, these primitives, along with a finite set of elements,
are all we need to define a finite cyclic group, which is all we need
to define the Diffie-Hellman function. Not important to understand the
abstract jargon, just FYI.
```

In [1]:
from challenge_31 import do_sha256, hmac
from challenge_39 import invmod
from challenge_57 import primegen, crt
from functools import reduce
from itertools import count
from operator import mul
from random import randrange
from math import log

In [2]:
class Zero:
    pass


class Curve:
    def __init__(self, a, b, p):
        self.zero = Zero()
        self.a = a
        self.b = b
        self.p = p
    
    def inv(self, pt):
        x, y = pt
        p = self.p
        return (x, p-y)  # = (x, -y) in GF(p)

    def add(self, p1, p2):  # don't worry about how this works. it's ~magic~
        zero = self.zero
        if p1 is zero: return p2
        if p2 is zero: return p1
        if p1 == self.inv(p2): return zero
        
        a, p = self.a, self.p
        x1, y1 = p1
        x2, y2 = p2
        
        if p1 == p2:
            top = (3 * x1**2 + a) % p
            btm = (2 * y1) % p
        else:
            top = (y2 - y1) % p
            btm = (x2 - x1) % p
        m = (top * invmod(btm, p)) % p
        
        x3 = (m**2 - x1 - x2) % p
        y3 = (m*(x1 - x3) - y1) % p
        return x3, y3
    
    def mul(self, pt, k):
        result = self.zero
        add = self.add
        while k:
            if k & 1:
                result = add(result, pt)
            pt = add(pt, pt)
            k >>= 1
        return result

```
Let's put this newfound knowledge into action. Implement a set of
functions up to and including elliptic curve scalar
multiplication. (Remember that all computations are in GF(p), i.e. mod
p.) You can use this curve:

    y^2 = x^3 - 95051*x + 11279326

Over GF(233970423115425145524320034830162017933). Use this base point:

    (182, 85518893674295321206118380980485522083)

It has order 29246302889428143187362802287225875743.

Oh yeah, order. Finding the order of an elliptic curve group turns out
to be a bit tricky, so just trust me when I tell you this one has
order 233970423115425145498902418297807005944. That factors to 2^3 *
29246302889428143187362802287225875743.

If your implementation works correctly, it should be easy to verify:
remember that multiplying the base point by its order should yield the
group identity.
```

In [3]:
curve = Curve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)

base = (182, 85518893674295321206118380980485522083)
order = 29246302889428143187362802287225875743

# Quick test: make sure the base point times the order equals the group's identity element
assert curve.mul(base, order) is curve.zero

```
Implement ECDH and verify that you can do a handshake correctly. In
this case, Alice and Bob's secrets will be scalars modulo the base
point order and their public elements will be points. If you
implemented the primitives correctly, everything should "just work".
```

In [4]:
class ECDHKeypair:
    _priv = None
    pub = None
    
    def __init__(self, curve):
        self.curve = curve
        self.keygen()
        
    def keygen(self):
        curve = self.curve
        
        priv = randrange(0, order)
        pub = curve.mul(base, priv)
        
        self._priv = priv
        self.pub = pub
    
    def handshake(self, other_pub):
        return self.curve.mul(other_pub, self._priv)

In [5]:
# Let's run through a test handshake to make sure our ECDH implementation is sound.
# We'll encapsulate this test in a function to avoid polluting the top-level namespace.

def test_handshake():
    alice = ECDHKeypair(curve)
    bob = ECDHKeypair(curve)

    alice_secret = alice.handshake(bob.pub)
    bob_secret = bob.handshake(alice.pub)

    print("Alice's version of shared secret:", alice_secret)
    print("Bob's version of shared secret:  ", bob_secret)
    assert alice_secret == bob_secret
    print("ECDH handshake successful!")
    
test_handshake()

Alice's version of shared secret: (190505261385647865938462404279859891977, 31017764830082533994147172335899137503)
Bob's version of shared secret:   (190505261385647865938462404279859891977, 31017764830082533994147172335899137503)
ECDH handshake successful!


```
Next, reconfigure your protocol from #57 to use ECDH.
```

In [6]:
def point_to_bytes(pt):
    # helper function
    # this is a quick & dirty hack for turning an EC point into something we can hash
    # assumes log(p, 2) < 128
    if isinstance(pt, Zero):
        pt = (0, 1)
    return pt[0].to_bytes(128, 'big') + pt[1].to_bytes(128, 'big')
assert log(curve.p, 2) < 128


def bob_coro(message=b'crazy flamboyant for the rap enjoyment', curve=curve):
    keypair = ECDHKeypair(curve)
    print("Bob's secret private key:", keypair._priv)

    # announce our public key on coroutine initialization (before generating first response)
    output = keypair.pub
    
    while True:
        remote_pub = (yield output)
        secret = keypair.handshake(remote_pub)
        mac_key = do_sha256(point_to_bytes(secret))
        mac = hmac(mac_key, message)
        output = (message, mac)

In [7]:
# quick test: make sure Bob gives us correct MACs and doesn't throw any errors

def test_bob(n=10):
    bob = bob_coro()
    bob_pub = next(bob)

    for _ in range(n):
        keypair = ECDHKeypair(curve)
        message, mac = bob.send(keypair.pub)

        mac_key = do_sha256(point_to_bytes(keypair.handshake(bob_pub)))
        assert hmac(mac_key, message) == mac

    print("Bob appears to be working!")
    
test_bob()

Bob's secret private key: 27908841264252045078566735712826485678
Bob appears to be working!


```
Can we apply the subgroup-confinement attacks from #57 in this
setting? At first blush, it seems like it will be pretty difficult,
since the cofactor is so small. We can recover, like, three bits by
sending a point with order 8, but that's about it. There just aren't
enough small-order points on the curve.

How about not on the curve?

Wait, what? Yeah, points *not* on the curve. Look closer at our
combine function. Notice anything missing? The b parameter of the
curve is not accounted for anywhere. This is because we have four
inputs to the calculation: the curve parameters (a, b) and the point
coordinates (x, y). Given any three, you can calculate the fourth. In
other words, we don't need b because b is already baked into every
valid (x, y) pair.

There's a dangerous assumption there: namely, that the peer will
submit a valid (x, y) pair. If Eve can submit an invalid pair, that
really opens up her play: now she can pick points from any curve that
differs only in its b parameter. All she has to do is find some curves
with small subgroups and cherry-pick a few points of small
order. Alice will unwittingly compute the shared secret on the wrong
curve and leak a few bits of her private key in the process.

How do we find suitable curves? Well, remember that I mentioned
counting points on elliptic curves is tricky. If you're very brave,
you can implement Schoof-Elkies-Atkins. Or you can use a computer
algebra system like SageMath. Or you can just use these curves I
generated for you:

y^2 = x^3 - 95051*x + 210

y^2 = x^3 - 95051*x + 504

y^2 = x^3 - 95051*x + 727

They have orders:

233970423115425145550826547352470124412

233970423115425145544350131142039591210

233970423115425145545378039958152057148

They should have a fair few small factors between them. So: find some
points of small order and send them to Alice. You can use the same
trick from before to find points of some prime order r. Suppose the
group has order q. Pick some random point and multiply by q/r. If you
land on the identity, start over.

It might not be immediately obvious how to choose random points, but
you can just pick an x and calculate y. This will require you to
implement a modular square root algorithm; use Tonelli-Shanks, it's
pretty straightforward.

Implement the key-recovery attack from #57 using small-order points
from invalid curves.
```

In [8]:
from challenge_59 import tonelli_shanks, find_point_of_order_r

def test_tonelli_shanks():
    p = 17
    for i in range(1, p):
        sq = pow(i, 2, p)
        roots = tonelli_shanks(sq, p)
        assert i in roots
    print("Tonelli-Shanks appears to be working!")
test_tonelli_shanks()

Tonelli-Shanks appears to be working!


In [9]:
# Parameters for our new curves:
b_vals = [210, 504, 727]
new_orders = [233970423115425145550826547352470124412,
              233970423115425145544350131142039591210,
              233970423115425145545378039958152057148]

In [10]:
bob = bob_coro()
bob_pub = next(bob)
bob_pub  # bob's public key

Bob's secret private key: 24929733335361946273336756535757847696


(93925966875889713149384376576371996816,
 141028857612932217596051080106443748707)

In [11]:
moduli = []
residues = []

for b_val, new_order in zip(b_vals, new_orders):
    new_curve = Curve(curve.a, b_val, curve.p)
    
    print("\nNow using b =", b_val)
    print("Partially factoring curve's order...")

    small_non_repeated_factors = [p for p in primegen(up_to=2**16)
                                  if new_order % p == 0]

    divisors = [d for d in small_non_repeated_factors
                if d not in moduli and d != 2]  # d=2 gives us points with y-coord 0 - more trouble than it's worth

    moduli += divisors
    
    if divisors:
        print("New moduli:", divisors)
        print("Gathering residues...")
        for d in divisors:
            base_pt = find_point_of_order_r(d, new_curve, new_order)
            message, mac = bob.send(base_pt)
            
            # run exhaustive search on range(d) to determine bob._priv % d
            pt = curve.zero
            for i in range(d):
                mac_key = do_sha256(point_to_bytes(pt))
                if hmac(mac_key, message) == mac:
                    break
                pt = curve.add(base_pt, pt)
            else:
                raise Exception("couldn't find mac key")
            
            residues.append(i)  # i = bob._priv % d
        print("Done.")

assert reduce(mul, moduli, 1) > curve.p  # make sure we have enough moduli for the CRT to work
print("\nWe have enough data to use the CRT!")


Now using b = 210
Partially factoring curve's order...
New moduli: [3, 11, 23, 31, 89, 4999, 28411, 45361]
Gathering residues...
Done.

Now using b = 504
Partially factoring curve's order...
New moduli: [5, 7, 61, 12157, 34693]
Gathering residues...
Done.

Now using b = 727
Partially factoring curve's order...
New moduli: [37, 67, 607, 1979, 13327, 13799]
Gathering residues...
Done.

We have enough data to use the CRT!


In [12]:
print("Residues:", residues)
print("Moduli:", moduli)

res = crt(residues, moduli)  # this is our guess for Bob's private key
assert res[1] > curve.p
bob_priv = res[0]
bob_pub_derived = curve.mul(base, bob_priv)

print()
print("Bob's private key (derived):", bob_priv)
print("Bob's public key (derived): ", bob_pub_derived)
print("Bob's public key (actual):  ", bob_pub)
assert bob_pub == bob_pub_derived
print("\n💥 Bob's ECDH private key has been recovered.")

Residues: [1, 2, 7, 11, 75, 2635, 1405, 38061, 1, 1, 8, 5290, 24038, 17, 20, 110, 446, 7736, 4264]
Moduli: [3, 11, 23, 31, 89, 4999, 28411, 45361, 5, 7, 61, 12157, 34693, 37, 67, 607, 1979, 13327, 13799]

Bob's private key (derived): 24929733335361946273336756535757847696
Bob's public key (derived):  (93925966875889713149384376576371996816, 141028857612932217596051080106443748707)
Bob's public key (actual):   (93925966875889713149384376576371996816, 141028857612932217596051080106443748707)

💥 Bob's ECDH private key has been recovered.
